# Community detection

<font size="4">which journal(s) initiate(s) chains of circulations outside the AG community</font>

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import sparql_dataframe
import geopandas as gpd
from shapely import wkt
import matplotlib.pyplot as plt
import folium
import VizKG.visualize as vkg
import plotly.express as px

In [2]:
endpoint = "http://localhost:3030/VisualContagions/sparql"

<font size="4">To understand the impact of the AG in other types of publications/communities, we search for clusters that have both images from AG journals as well as images NOT from AG journals</font>

In [6]:
q ="""

PREFIX crmdig: <http://www.ics.forth.gr/isl/CRMdig/>
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX aat: <http://vocab.getty.edu/aat/>


SELECT DISTINCT ?cluster ?identifier ?AGJournal ?notAGJournal ?JournalTypeLabel ?image ?image1 WHERE {

  
	VALUES ?avantgarde { <https://lod.unige.ch/vc/type/988EE2E2-172C-3104-8DAA-B4D629E814AE> } .
	

		?cluster crm:P148_has_component ?image ;
           	crm:P1_is_identified_by ?identifier .
		?image crm:P128i_is_carried_by ?issue . 
		?issue crm:P2_has_type ?avantgarde;
				rdfs:label ?AGJournal.  


	
		?cluster crm:P148_has_component ?image1 . 
		?image1 crm:P128i_is_carried_by ?issue1 . 
		?issue1 crm:P2_has_type ?type ;
				rdfs:label ?notAGJournal .
		?type rdfs:label ?JournalTypeLabel .

		FILTER (contains(str(?type), "https://lod.unige.ch/vc/type/"))
		FILTER (?type != ?avantgarde)

# Trashy cluster not useful for the analysis

	FILTER NOT EXISTS {
                          VALUES ?trash { <https://visualcontagions.unige.ch/explore/duplicates/clusters/0000030040d2720fb2fd94a38df5dc5b0d55e14309962e6c764e23a54a819816cdfe9c20> <https://visualcontagions.unige.ch/explore/duplicates/clusters/000015004c44ed7f0e7150811be78d4b930ef72322476f4d9cadcf1f73b08117bae435a1> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0000e8009061dc5b29fb17872c9c13ac1a64fe42e8b94af2481e59886f28d0156fde0ceb> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00026200b3d151e5c855c651b2a0bea2c9116be1a5e5b7141bacc846b12d3708338ff615> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0000ca00a432dfdd48c6534088f51065c6f2983133e467314d5d6a4560ae5ea16de6f359> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0000e8009061dc5b29fb17872c9c13ac1a64fe42e8b94af2481e59886f28d0156fde0ceb> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00008a005e931cc1f1dc0dc0f82d3ee457c5b642600420ae9313b50ad6c0d75b41b8f1c0> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0001bd00a370039869e9e1bb229cd73ddc231b51110e135de82d755ef72e3b367a5411e9> <https://visualcontagions.unige.ch/explore/duplicates/clusters/000109002f020235176a28018a0944ed84b4b7ddbed8737474caaa5c0b54922a4cd6073c> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00024100820c145c252454de492f79784d2c1df4b3a13d9eabf0179f0e0305b516a764ba> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0003ec00036e6aa8355e0f69196bf6006defd506c93e04722b54cf7831c66c3c3b8dbb05> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00000200f89ef4fb7b639ce5e811ec9ec425c3da3b92dd5edb5d9e64fbf8d010f9d96d71> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0000040066b73cf2a0b0742a0aceecc4dd1724b52dc7afbdc4b48432ee3f0336124798aa> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00001200bed2a9ade1689744b661b58c1d93bcd442cbeafcf07b6460d0df1b33d65f5b86> <https://visualcontagions.unige.ch/explore/duplicates/clusters/000022005d01a84e38421213b8816fd78e4aef0890445f65f81de2ee1459980a6335942a> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00001600fe63944181de71b69fc3900867dc30611de913a214c5d42ac2e7ce3dd7cff99f> <https://visualcontagions.unige.ch/explore/duplicates/clusters/000012009d66187e47b439dd2c2e6668dd9945209f40685c7039c56c563a4aa9bbfa9718> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00000d0069b0d3bd802ae948c1f53b96391afeb46fcd505e999cb56cd37406508e4ef418> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00000b002057ee6db2a8ea72c04e36379e7b035a4d2911a1155b5652230551866825ab90> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00000b0003d9d58d012451307b2ef5fbd75af31eec0c5f1afcdefb10b4ab8fbc38d15615> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00000a00609f4e0b53f5b2ca7afaea13c7a3ba187b05ae3bdd1108598e5226a7aaf18ded> <https://visualcontagions.unige.ch/explore/duplicates/clusters/000008005b033c554cfd743ed238cd05b35f46fe7de56720622971b00aecaf985896fe4a> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00000600d4e85eaf39fb676e999b8d2466a4455d48d70791ff2d4dd1cc4653c248264df3> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00002a00a60bf43cf1fb052165221ee1727e977b2ecf99f820cbbd426d12c6aa6a37d9e4> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0000310098549f748176d2f4efedf76773dca2aafeaf8db5f746daf37a50dd3d0819dbb3> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0003d500d8362a66c04fe8273b00d9470929fd918a6dc6c4395f060e0c5e270f12f54f5c> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0003ff00a6da95a6b61b8c47220d9839b7b4b6afe26daad02ee0c0c291c40ad1bb0bec90> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0000c9000f961bd877cdc346bd51b76a74b698924ae6960e3bbfa0986e7aea7da730930c> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0000d000ecf8a0329d2530d3eb601e077c0e5c298db100b8316c0af9232e41e8a7e8e4ef> <https://visualcontagions.unige.ch/explore/duplicates/clusters/000403008f37a6852f6339eefe0e74225b20553b5d641b32015b15d40ca4d216a540ae2f> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00002400d7b4ea535865b256e1c13f0d6505182ead425183c8f919bfa3309aab1dae43dc> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00001f00c18ad27c34a66610ece997934ce4bc1aa5fbcdad562e58a52ab86792352bfb9c> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00002b000918272590bc01401e6367c6448a222445fe13cabec0a2a38a7b38e598cea622> <https://visualcontagions.unige.ch/explore/duplicates/clusters/000057008d59ea30b6175a9c5c32ebf3caa2c27cce0d569fd3b2a899ebf0107086c3a197> <https://visualcontagions.unige.ch/explore/duplicates/clusters/000028007bf1c23529418f9fba6de2955caa8fa4e1572632151c41a9f1b20c1c5a45a1c5> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00005a009d47a1932fcbbe7ff04ac3017fa6d167f06fe3287477d6c1cedae50451cd67dc> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00001f0087307813ef1d16a6def5d96b8b6550a707da4fd6702aa64d8425b29e6826f049> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00001c0000968f38146238bab2f376c40b0fca62e734f151a85e2ecce4981e0050eb57a0> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00001d00ffc088f4f254a86d70e0a3495bbf5f288b1373e96c6875d487cd1b14076e2702> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00001d000d01d183fa765191f6028c4b69a7e1c9c9ed1d9a3ae14bb9a4731b5172f4b337> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00001c002016002f400721bfa0a32f16e887e64cc4e092cd626a7fb7c426545e0757e7f1> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0002910088db3f58b1c573c281f266c695c21d421c9337abca57d4a51440e13a2f9c5893> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00028200ce1d42e579bb46d53b1b038b5f23bf88a8057f5d5cb9806d9c87dee28477ecc4>
                                       } .
    ?cluster crm:P1_is_identified_by ?trash . } 

}



"""

In [7]:
df = sparql_dataframe.get(endpoint, q)

In [8]:
df

,cluster,identifier,AGJournal,notAGJournal,JournalTypeLabel,image,image1
0,https://lod.unige.ch/vc/cluster/00000500e18f2d...,https://visualcontagions.unige.ch/explore/dupl...,Dau al Set,Le Cri de Toulouse,News,https://iiif.unige.ch/iiif/2/fedora_ug8079003;...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k53...
1,https://lod.unige.ch/vc/cluster/00000500e18f2d...,https://visualcontagions.unige.ch/explore/dupl...,Dau al Set,Le Cri de Toulouse,News,https://iiif.unige.ch/iiif/2/fedora_ug8079003;...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k53...
2,https://lod.unige.ch/vc/cluster/00000200825250...,https://visualcontagions.unige.ch/explore/dupl...,Noi,Der Cicerone,Modern Art Journal,https://iiif.unige.ch/iiif/2/fedora_ug8088904;...,https://digi.ub.uni-heidelberg.de/iiif/2/cicer...
3,https://lod.unige.ch/vc/cluster/human/296,https://visualcontagions.unige.ch/explore/anal...,Umělecký mĕsíčník,La Revue de l'art ancien et moderne,Art History,https://bluemountain.princeton.edu/bluemtn/iii...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k43...
4,https://lod.unige.ch/vc/cluster/human/296,https://visualcontagions.unige.ch/explore/anal...,Umělecký mĕsíčník,L'Art vivant (Paris. 1925),Modern Art Journal,https://bluemountain.princeton.edu/bluemtn/iii...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k30...
...,...,...,...,...,...,...,...
34951,https://lod.unige.ch/vc/cluster/human/223,https://visualcontagions.unige.ch/explore/anal...,La Révolution surréaliste,Arts Magazine,Art Journal,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k15...,https://iiif.archivelab.org/iiif/sim_arts-maga...
34952,https://lod.unige.ch/vc/cluster/human/223,https://visualcontagions.unige.ch/explore/anal...,La Révolution surréaliste,L'Amour de l'art (1920),Modern Art Journal,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k15...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k42...
34953,https://lod.unige.ch/vc/cluster/human/223,https://visualcontagions.unige.ch/explore/anal...,La Révolution surréaliste,Deutsche Kunst und Dekoration,Decorative Art,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k15...,https://digi.ub.uni-heidelberg.de/iiif/2/dkd19...
34954,https://lod.unige.ch/vc/cluster/human/223,https://visualcontagions.unige.ch/explore/anal...,La Révolution surréaliste,Der Cicerone,Modern Art Journal,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k15...,https://digi.ub.uni-heidelberg.de/iiif/2/cicer...


In [ ]:
df['count_cluster'] = df.groupby(['identifier'])['cluster'].transform('count')
df.sort_values(by=['identifier'], ascending=False, inplace=True)

In [ ]:
df['nOfJournalTypes'] = df.groupby(['JournalTypeLabel'])['notAGJournal'].transform('count')
df['count_notAGJournal'] = df.groupby(['notAGJournal'])['JournalTypeLabel'].transform('count')

In [ ]:
df

In [ ]:
df.to_csv('csv/community.csv', index=False)

In [ ]:
df_above_20 = df[df['count_cluster'] >= 20]
df_above_20 = df_above_20.drop_duplicates(subset='identifier', keep="first")
df_above_20.sort_values(by=['count_cluster'], ascending=False, inplace=True)
df_above_20.to_csv('csv/df_above_20.csv', index=False)

In [ ]:
df_below_50 = df[df['count_cluster'] <= 50]

In [ ]:
df_below_50

In [ ]:
df_below_50['count_community'] = df_below_50.groupby(['JournalTypeLabel'])['notAGJournal'].transform('count')

In [ ]:
df_below_50['count_notAGJournal'] = df_below_50.groupby(['notAGJournal'])['JournalTypeLabel'].transform('count')

In [ ]:
df_below_50['count_AGJournal'] = df_below_50.groupby(['AGJournal'])['identifier'].transform('count')

In [ ]:
df_below_50.to_csv('community_below_50.csv', index=False)

In [ ]:
df_viz = df_below_50[df_below_50['count_community'] >= 50]

fig = px.bar(df_viz, y='count_community', x='JournalTypeLabel', color='notAGJournal',
            title="Default")
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
#fig.show()
fig.write_html('community.html', auto_open=True)

In [ ]:
fig = px.sunburst(df_viz, path=['JournalTypeLabel', 'notAGJournal'], values='count_community')
fig.update_layout(uniformtext=dict(minsize=10, mode='hide'))
#fig.show()
fig.write_html('community_sunburst.html', auto_open=True)